# Syntrophic interactions in a simplified anaerobic CO2-fixating microbial community
## Preliminaries

In [ ]:
import numpy as np
import pandas as pd
import cobra as cb
import micom as mc
import itertools
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import coco as co
from constraints import *
config = cb.Configuration()
config.solver = 'cplex'

Set up paths and variables

In [ ]:
sample_info_path = '../data/sample_info.tsv'
dna_path = '../data/all_MAGs_coverage.txt'
rna_path = '../data/RNA/'
gene_path = '../data/gene_length.csv'
model_path = '../models/gapseq/'
community_path = '../models/micom/'
media_path = '../data/media_db.tsv'
community_growth_path = '../data/growth_rates.csv'
biochemistry_path = '../data/feedstock.csv'
results_path = 'results/'
figures_path = 'figures/'
build_communities = True
abundance_threshold = 0.005

## Data preparation

Load MAG abundances

In [ ]:
dna_df = pd.read_table(dna_path, sep='\t', index_col='Bin Id')
mag_size_s = dna_df['Bin size (Mbp)']
dna_df = dna_df.iloc[:, dna_df.columns.str.contains('% community', regex=False)]
dna_df.columns = dna_df.columns.str.replace('.sorted: % community', '', regex=False)
dna_df.index = dna_df.index.str.replace('.fasta', '', regex=False)
dna_df = dna_df.loc[dna_df.index != 'unbinned', :]
MAG_names = dna_df.index

Load sample information and set column names

In [ ]:
sample_df = pd.read_table(sample_info_path, index_col='NGS ID')
dna_df.columns = [s[0] for s in dna_df.columns.str.split('S')]
dna_df.columns = sample_df.loc[dna_df.columns, 'ID']

Load RNA abundances

In [ ]:
df_list = []
key_list = []
for m in MAG_names:
    try:
        counts = pd.read_table(rna_path + m + '.csv_log2.csv', sep=',', index_col='Unnamed: 0')
        counts.index = counts.index.str.replace('.faa_', '__', regex=False)
        counts.index = counts.index.str.replace('pilon_pilon', '', regex=False)
        counts.index = counts.index.str.replace('_pilon', '', regex=False)
        df_list.append(counts)
        key_list.append(m)
    except:
        pass
rna_df = pd.concat(df_list, axis=0, keys=key_list)
# Set column names
rna_df.columns = rna_df.columns.str.replace('1_[ABC][123]', '\g<0>-s2', regex=True)
rna_df.columns = rna_df.columns.str.replace('2_[ABC][123]', '\g<0>-s3', regex=True)
rna_df.columns = rna_df.columns.str.replace('Read_Count_[12]_', '', regex=True)

Aggregate and normalise RNA abundances

In [ ]:
# Sum RNA abundances and log-transform
tot_rna_df = pd.DataFrame(index=MAG_names, columns=rna_df.columns)
for s in rna_df.columns:
    count_sums = rna_df[s].groupby(level=0).sum()
    tot_rna_df.loc[:, s] = np.log(count_sums)
# Load gene lengths
gene_df = pd.read_table(gene_path, sep=',', index_col=None)
gene_df['gene'] = gene_df['gene'].str.replace('.faa_', '__', regex=False)
gene_df['gene'] = gene_df['gene'].str.replace('pilon_pilon', '', regex=False)
gene_df['gene'] = gene_df['gene'].str.replace('_pilon', '', regex=False)
#gene_df = gene_df.loc[gene_df['gene'].isin(rna_df.index.get_level_values(1)), :]
gene_df['MAG'] = gene_df['gene'].str.split('__', expand=True)[0]
gene_df.set_index(['MAG', 'gene'], inplace=True)
# Sum gene lengths
tot_gene_df = gene_df['gene_length'].groupby(level=0).sum().div(1e6)
tot_gene_df = tot_gene_df.loc[tot_gene_df.index.isin(rna_df.index.get_level_values(0))]
# Normalise RNA abundances with total coding sequence length
tot_rna_df = tot_rna_df.divide(tot_rna_df.sum(axis=0))
tot_rna_df = tot_rna_df.divide(tot_gene_df, axis=0)
tot_rna_df = 100*tot_rna_df.divide(tot_rna_df.sum(axis=0))

Select the taxa above the abundance threshold

In [ ]:
# Select samples with both DNA and RNA
sample_names = tot_rna_df.columns
dna_df = dna_df.loc[:, sample_names]
# Combine nucleic acid abundances
na_df = dna_df.multiply(tot_rna_df)
na_df = na_df.divide(na_df.sum(axis=0))
# Filter 
na_df = na_df.loc[na_df.index != 'unbinned', :]
#na_df = na_df.loc[na_df.max(axis=1) >= abundance_threshold, :]
MAG_idx = na_df.max(axis=1) >= abundance_threshold
MAG_names = na_df.loc[MAG_idx, :].index
# Subset RNA tables
rna_df = rna_df.loc[MAG_names, :]
print('Selected', str(len(MAG_names)), 'MAGs:')
print(list(MAG_names))

In [ ]:
for s in sample_names:
    print(s, end=' ')

    # Filter species
    species_names = list(na_df.index[na_df[s] >= 0.005])
    community_abundance = (dna_df.loc[species_names, s]).sum()
    print(community_abundance)

Adjust MAG names for compatibility with Micom

In [ ]:
dna_df.index = dna_df.index.str.replace('.', '_', regex=False)
rna_df.index = pd.MultiIndex.from_arrays([rna_df.index.get_level_values(0).str.replace('.', '_', regex=False), \
    rna_df.index.get_level_values(1).str.replace('.', '_', regex=False)], names=('MAG', 'gene'))
na_df.index = na_df.index.str.replace('.', '_', regex=False)

In [ ]:
sample_names

Prepare the media

In [ ]:
media_df = pd.read_table(media_path, sep='\t', index_col=None)
media_df['excRxn'] = 'EX_' + media_df['met'] + '_m'
# dictionary establishing the medium in each sample
media_dict = dict(zip(sample_names, np.repeat(['BA[acetate+CO2]'], len(sample_names))))

Prepare the biochemical data

In [ ]:
biochemistry_df = pd.read_table(biochemistry_path, sep=',', index_col=None)
biochemistry_df = biochemistry_df.drop('metName', axis=1)
biochemistry_df['excRxn'] = 'EX_' + biochemistry_df['met'] + '_m'

## Build microbial community models

Create community models for each sample

In [ ]:
if build_communities:
    for s in sample_names:
        print(s)

        # Filter species
        species_names = list(na_df.index[na_df[s] >= abundance_threshold])
        abundances = list(dna_df.loc[species_names, s])
        model_paths = [a+b for a,b in zip([model_path]*len(species_names), species_names)]
        model_paths = [a+b for a,b in zip(model_paths, [".xml"]*len(species_names))]
        model_paths = pd.Series(model_paths).str.replace('metabat1_', 'metabat1.')
        model_paths = model_paths.str.replace('metabat2_', 'metabat2.')
        model_paths = model_paths.str.replace('maxbin2_', 'maxbin2.')
        model_paths = model_paths.str.replace('vamb_', 'vamb.')
        model_paths = model_paths.str.replace('metabat1.31', 'metabat1.31-curated')
        model_paths = list(model_paths.str.replace('metabat1.59', 'metabat1.59-curated'))
        print(species_names)

        # Build community model
        species_df = pd.DataFrame(data={'id': species_names, 'file': model_paths, 'abundance': abundances})
        com = mc.Community(species_df, solver='cplex')
        print("Built a community with a total of {} reactions.".format(len(com.reactions)))
        print("Built a community with a total of {} metabolites.".format(len(com.metabolites)))

        # Save community model
        com.to_pickle(community_path + s + '_' + str(abundance_threshold) + '.pickle')

## Parameter exploration
Explore parameters for CoCo modelling. 

First, initialise CoCo

In [ ]:
cocoGEM_builder = co.CoCo(rna_df, sparse_community=True)

Parameter exploration

In [ ]:
deltas = [0.5, 0.6, 0.7, 0.8, 0.9, 1., 2., 3., 4., 5.] #np.arange(1., 11., 1.)
gammas = [1] #np.arange(1., 3)
tradeoffs = np.round(np.arange(0.1, 1.05, 0.1), 1) # cooperative tradeoff fraction
label = ''
method='ct'
verbose=False

param_product_bound, delta_bound = cocoGEM_builder.get_param_range()
combinations = np.array([c for c in itertools.product(deltas, gammas, tradeoffs)])
param_products = np.array([i[0]*i[1] for i in combinations])
combinations = list(map(tuple, combinations[param_products <= param_product_bound]))
community_dict = {} # every element is a list of species for a condition
exp_growth = []
abundances = []
base_df_list = []
coco_df_list = []
base_community_growth = pd.DataFrame(index=sample_names, columns=pd.MultiIndex.from_tuples(combinations))
coco_community_growth = pd.DataFrame(index=sample_names, columns=pd.MultiIndex.from_tuples(combinations))

print('deltas =', deltas)
print('gammas =', gammas)
print('tradeoffs =', tradeoffs)
print('method =', method)


for s in sample_names:
    print('----------------------------------')
    print('>\t', s)
    print('----------------------------------')

    # Filter species
    species_names = list(na_df.index[na_df[s] >= abundance_threshold])
    abundances.append(list(dna_df.loc[species_names, s]))
    community_dict[s] = species_names
    community_abundance = (na_df.loc[na_df[s] >= abundance_threshold, s]).sum()
    print(species_names)

    # Load community model
    com = mc.load_pickle(community_path + s + '_' + str(abundance_threshold) + '.pickle')
    print("Built a community with a total of {} reactions.".format(len(com.reactions)))
    print("Built a community with a total of {} metabolites.".format(len(com.metabolites)))

    # fixes
    if 'metabat1_59' in species_names:
        com.reactions.rxn90070_c0__metabat1_59.bounds = 0.0, 1000.0
        com.reactions.rxn90072_c0__metabat1_59.bounds = -1000.0, 0.0
    
    # apply environmental constraints
    com = set_medium(com, s, media_dict, media_df)
    com = set_biochemical_constraints(com, s, biochemistry_df, community_abundance)
    
    # gene coverage check
    _ = cocoGEM_builder.check_gene_coverage(com, verbose=True)

    # simulations over delta, gamma and tradeoff
    base_growths = pd.DataFrame(index=species_names, columns=pd.MultiIndex.from_tuples(combinations))
    coco_growths = pd.DataFrame(index=species_names, columns=pd.MultiIndex.from_tuples(combinations))
    for t in tradeoffs:
        try:
            if method == 'ct':
                sol = com.cooperative_tradeoff(fraction=t, fluxes=False, min_growth=0.01)
            elif method == 'fba':
                sol = com.optimize_all()
            for c in combinations:
                if c[2] == t:
                    if method == 'ct':
                        base_growths.loc[:, c] = list(sol.members.loc[species_names, 'growth_rate'])
                        base_community_growth.loc[s, c] = sol.growth_rate
                    elif method == 'fba':
                        base_growths.loc[:, c] = list(sol.loc[species_names])
        except:
            print('No solution')
    print('-----------------')

    for c in combinations:
        with com as coco_com:
            print('delta = ' + str(c[0]) + ', gamma = ' + str(c[1]) + ', tradeoff = ' + str(c[2]))
            cocoGEM_builder.build(coco_com, s, c[0], c[1], verbose=verbose)
            print('-----------------')
            try:
                if method == 'ct':
                    sol = coco_com.cooperative_tradeoff(fraction = c[2], fluxes=False)
                    coco_growths.loc[:, c] = list(sol.members.loc[species_names, 'growth_rate'])
                    coco_community_growth.loc[s, c] = sol.growth_rate
                elif method == 'fba':
                    sol = coco_com.optimize_all()
                    coco_growths.loc[:, c] = list(sol.loc[species_names])
            except:
                print('No solution')
    base_df_list.append(base_growths)
    coco_df_list.append(coco_growths)

    print('\n')
print('Finished')

# wrap up results
base_growth = pd.concat(base_df_list, axis=0, keys=sample_names)
coco_growth = pd.concat(coco_df_list, axis=0, keys=sample_names)
abundances = pd.Series(np.hstack(abundances))

# save
base_growth.to_csv(results_path + 'base_growths_' + label + '.csv')
coco_growth.to_csv(results_path + 'coco_growths_' + label + '.csv')
base_community_growth.to_csv(results_path + 'base_community_growths_' + label + '.csv')
coco_community_growth.to_csv(results_path + 'coco_community_growths_' + label + '.csv')
abundances.to_csv(results_path + 'abundances.csv')

Extract the parameters yielding the best correlations with community growth

In [ ]:
exp_community_growth = pd.read_table(community_growth_path, sep=',', index_col='sample')
exp_community_growth = exp_community_growth.T.squeeze()
exp_community_growth

In [ ]:
from scipy.stats import pearsonr

label = ''

base_growth_df = base_community_growth.drop(['A2-s3'], axis=0)
coco_growth_df = coco_community_growth.drop(['A2-s3'], axis=0)
exp_growth_s = exp_community_growth.drop(['A2-s3'], axis=0)

base_growth_df[base_growth_df.abs() < 1e-6] = 0
coco_growth_df[coco_growth_df.abs() < 1e-6] = 0

# determine the best correlations
base_corrs = pd.DataFrame(index=['pearsonr','pvalue'], columns=base_growth_df.columns)
coco_corrs = pd.DataFrame(index=['pearsonr','pvalue'], columns=coco_growth_df.columns)
idx = np.array(np.isnan(exp_growth_s))
for c in base_growth_df.columns:
    if sum(base_growth_df.loc[:, c].isna()) == 0:
        base_corrs.loc[:, c] = pearsonr(exp_growth_s[~idx], base_growth_df.loc[~idx, c])
for c in coco_growth_df.columns:
    if sum(coco_growth_df.loc[:, c].isna()) == 0:
        coco_corrs.loc[:, c] = pearsonr(exp_growth_s[~idx], coco_growth_df.loc[~idx, c])
best_base_idx = pd.to_numeric(base_corrs.loc['pearsonr']).idxmax()
best_coco_idx = pd.to_numeric(coco_corrs.loc['pearsonr']).idxmax()
best_corrs = [base_corrs.loc['pearsonr', best_base_idx], \
    coco_corrs.loc['pearsonr', best_coco_idx]]
best_pvals = [base_corrs.loc['pvalue', best_base_idx], \
    coco_corrs.loc['pvalue', best_coco_idx]]

# format data
plot_data = pd.DataFrame(data={'Replication rate [a.u.]': list(exp_growth_s) + list(exp_growth_s), \
    'Growth rate [1/h]': np.hstack((base_growth_df.loc[:, best_base_idx], coco_growth_df.loc[:, best_coco_idx])),
    'Models': np.array(['Co-GEM']*len(exp_growth_s) + ['CoCo-GEM']*len(exp_growth_s)),
    'Conditions': np.tile(np.array(['A']*3 + ['B']*3 + ['C']*3 + ['A']*2 + ['B']*3 + ['C']*3), 2)})
plot_data['Growth rate [1/h]'] = plot_data['Growth rate [1/h]'].astype('float')

sns.set_style('ticks')
mpl.rcParams['axes.labelsize'] = 8
mpl.rcParams['xtick.labelsize'] = 8
mpl.rcParams['ytick.labelsize'] = 8
mpl.rcParams['axes.linewidth'] = 0.8
mpl.rcParams['font.sans-serif'] = 'Arial'

p = sns.lmplot(data=plot_data.loc[plot_data['Models']=='CoCo-GEM', :], x='Replication rate [a.u.]', y='Growth rate [1/h]', \
    height=1.8, aspect=1.2, scatter_kws={'edgecolors': 'none', 'facecolor': 'slategray', 'linewidths': 0}, line_kws={'color': 'slategray', 'linewidth': 0.8})
p.axes[0][0].set_xlim([0, 0.005])
p.axes[0][0].set_ylim([0.03, 0.1])
p.axes[0][0].set_xlabel('Measured growth rate [1/h]')
p.axes[0][0].set_ylabel('Model growth rate [1/h]')
p.axes[0][0].tick_params(axis='both', length=2)

# set scientific notation on the y axes
p.axes[0][0].ticklabel_format(style='scientific', scilimits=(0,0), axis='x')

text_kwargs = dict(ha='center', va='center', fontsize=6)
plt.text(0.002, 0.09, f"r = {str(np.around(best_corrs[1], 2))} (p = {str(np.around(best_pvals[1], 4))})", **text_kwargs)

p.fig.savefig(figures_path + 'growth_correlation.pdf', bbox_inches='tight')

## Flux calculation

In case CoCo is not already initialised

In [ ]:
cocoGEM_builder = co.CoCo(rna_df, sparse_community=True)

Calculate the full flux solutions for the best parameter combination

In [ ]:
delta = 0.9
gamma = 1.
base_tradeoff = 0.2
coco_tradeoff = 0.9
label = ''
method = 'fba' # 'fba', 'pfba'
considered_samples = list(sample_names[0:9])
verbose = False

community_dict = {} # every element is a list of species for a condition
base_flux_df_list = []
coco_flux_df_list = []
base_exc_df_list = []
coco_exc_df_list = []

print('delta =', delta)
print('gamma =', gamma)
print('base_tradeoff =', base_tradeoff)
print('coco_tradeoff =', coco_tradeoff)
print('method =', method)
print('considered_samples =', considered_samples)

for s in considered_samples:
    print('----------------------------------')
    print('>\t', s)
    print('----------------------------------')

    # Load community model
    com = mc.load_pickle(community_path + s + '_' + str(abundance_threshold) + '.pickle')
    species_names = list(com.taxonomy['id'])
    community_dict[s] = species_names
    rxns = [r.id for r in com.reactions]
    excRxns = [e.id for e in com.exchanges]
    community_abundance = (na_df.loc[na_df[s] >= abundance_threshold, s]).sum()
    print(species_names)
    print("Built a community with a total of {} reactions.".format(len(com.reactions)))
    print("Built a community with a total of {} metabolites.".format(len(com.metabolites)))

    # fixes
    if 'metabat1_59' in species_names:
        com.reactions.rxn90070_c0__metabat1_59.bounds = 0.0, 1000.0
        com.reactions.rxn90072_c0__metabat1_59.bounds = -1000.0, 0.0
        com.reactions.rxn90070_c0__metabat1_59.gene_reaction_rule = \
            '( metabat1_59__contig889_2 or metabat1_59__contig889_1239 or metabat1_59__contig889_617 ) and ( metabat1_59__contig889_3 or metabat1_59__contig889_624 )'
        com.reactions.rxn90072_c0__metabat1_59.gene_reaction_rule = \
            '( metabat1_59__contig889_2 or metabat1_59__contig889_1239 or metabat1_59__contig889_617 ) and ( metabat1_59__contig889_3 or metabat1_59__contig889_624 )'
    if 'metabat1_31' in species_names:
        com.reactions.rxn90070_c0__metabat1_31.gene_reaction_rule = \
            '( metabat1_31__contig81_166 or metabat1_31__contig81_167 )'
        com.reactions.rxn90072_c0__metabat1_31.gene_reaction_rule = \
            '( metabat1_31__contig81_166 or metabat1_31__contig81_167 )'
    
    # apply environmental constraints
    com = set_medium(com, s, media_dict, media_df)
    com = set_biochemical_constraints(com, s, biochemistry_df, community_abundance)
    
    # gene coverage check
    _ = cocoGEM_builder.check_gene_coverage(com, verbose=True)

    # simulations
    print('--- Base ---')
    with com:
        try:
            sol = com.cooperative_tradeoff(fraction = base_tradeoff, fluxes=False)
            med = mc.media.minimal_medium(com, 0.98*sol.growth_rate, min_growth=0.98*sol.members.growth_rate.drop("medium"))
            com.medium = med
        except:
            print('No growth solution')
        try:
            print('Calculating...')
            sol = com.cooperative_tradeoff(fraction = base_tradeoff, fluxes=True, pfba=(method == 'pfba'))
        except:
            sol = None
            print('No solution')
        if sol is not None:
            print(str(sol.growth_rate))
            base_flux_df_list.append(sol.fluxes.loc[species_names, :])
            base_exc_df_list.append(sol.fluxes.loc['medium', excRxns])
        else:
            base_flux_df_list.append(pd.DataFrame(index=species_names, columns=rxns))
            base_exc_df_list.append(pd.Series(index=excRxns))

    print('--- CoCo ---')
    with com as coco_com:
        cocoGEM_builder.build(coco_com, s, delta, gamma, meta=True)
        
        try:
            coco_sol = coco_com.cooperative_tradeoff(fraction = coco_tradeoff, fluxes=False)
            med = mc.media.minimal_medium(coco_com, 0.98*coco_sol.growth_rate, min_growth=0.98*coco_sol.members.growth_rate.drop("medium"))
            coco_com.medium = med
        except:
            print('No growth solution')
        try:
            print('Calculating...')
            coco_sol = coco_com.cooperative_tradeoff(fraction = coco_tradeoff, fluxes=True, pfba=(method == 'pfba'))
        except:
            coco_sol = None
            print('No solution')
        if sol is not None:
            print(str(sol.growth_rate))
            coco_flux_df_list.append(coco_sol.fluxes.loc[species_names, :])
            coco_exc_df_list.append(coco_sol.fluxes.loc['medium', excRxns])
        else:
            coco_flux_df_list.append(pd.DataFrame(index=species_names, columns=rxns))
            coco_exc_df_list.append(pd.Series(index=excRxns))
print('Finished')

# wrap up
base_fluxes = pd.concat(base_flux_df_list, axis=0, keys=considered_samples)
coco_fluxes = pd.concat(coco_flux_df_list, axis=0, keys=considered_samples)
base_exchanges = pd.DataFrame(base_exc_df_list, index=considered_samples).T
coco_exchanges = pd.DataFrame(coco_exc_df_list, index=considered_samples).T

# save
base_fluxes.to_csv(results_path + 'base_fluxes_' + label + '.csv')
coco_fluxes.to_csv(results_path + 'coco_fluxes_' + label + '.csv')
base_exchanges.to_csv(results_path + 'base_exchanges_' + label + '.csv')
coco_exchanges.to_csv(results_path + 'coco_exchanges_' + label + '.csv')


Clean values

In [ ]:
base_fluxes[base_fluxes.abs() < 1e-6] = 0
coco_fluxes[coco_fluxes.abs() < 1e-6] = 0
base_exchanges[base_exchanges.abs() < 1e-6] = 0
coco_exchanges[coco_exchanges.abs() < 1e-6] = 0

Define useful variables

In [ ]:
considered_samples = sample_names[0:9]

# multiply fluxes by the corresponding MAG abundance
abundances = []
for s in considered_samples:
    species_names = list(na_df.index[na_df[s] >= abundance_threshold])
    abundances.append(list(dna_df.loc[species_names, s]))
abundances = pd.Series(np.hstack(abundances))
coco_cum_fluxes = coco_fluxes.loc[considered_samples, :].mul(np.transpose(np.tile(list(abundances/100), (coco_fluxes.shape[1], 1))))

A_samples = ['A1-s2', 'A2-s2', 'A3-s2']
B_samples = ['B1-s2', 'B2-s2', 'B3-s2']
C_samples = ['C1-s2', 'C2-s2', 'C3-s2']